# **Fast Neural Style Transfer**
A simple example of using [Fast-NST algorithm](https://arxiv.org/abs/1705.06830) from pre-trained model available on Tensorflow Hub. Training such model from scratch would take a lot of time.

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import tensorflow_hub as hub

**Important info about Google Colab:**

1. *If you are using Google Colab the data can be imported from google drive, before doing so, include the code below.*
2. *Images are loaded via openCV, there is a patch for Google Colab for imshow function. Then you have to use cv2_imshow instead of cv2.imshow*
3. *Enable the GPU runtime!*

In [ ]:
# from google.colab.patches import cv2_imshow
# from google.colab import drive
# drive.mount('/content/drive')

**Download the Arbitrary image stylization model**

In [ ]:
model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

## **1. Images preparation**

In [ ]:
IMG_WIDTH = 400
IMG_HEIGHT = 300

CONTENT_IMAGE_URL = "img/louvre-small.jpg"
STYLE_IMAGE_URL = "img/k3.jpg"

### **1.1 Image preparation helper functions:**

In [ ]:
def load_image(url, width, height, color_mode):
    image = cv2.imread(url)
    image = cv2.resize(image, (width, height))
    
    print(image.shape, "\n")
    if cv2_imshow:
        cv2_imshow(image)
    else:
        cv2.imshow(image)

    image = cv2.cvtColor(image, color_mode)
    image = np.reshape(image, ((1,) + image.shape))
    image = tf.image.convert_image_dtype(image, tf.float32)

    return image


def tensor_to_image(tensor):
    tensor = tensor * 255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor) > 3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]

    return cv2.cvtColor(tensor, cv2.COLOR_BGR2RGB)

### **1.2 Load images:**

In [ ]:
print("\nContent image:")
content_image = load_image(CONTENT_IMAGE_URL, IMG_WIDTH, IMG_HEIGHT, cv2.COLOR_BGR2RGB)

print("\nStyle image:")
style_image = load_image(STYLE_IMAGE_URL, IMG_WIDTH, IMG_HEIGHT, cv2.COLOR_BGR2RGB)

## **2. Generate the image**
It's much easier to use this network compared to normal NST as it's based on pre-trained features. Furthermore, we are using ready to use model so we don't have to configure anything (unfortunately we can't customize it for best results).

In [ ]:
stylized_image = model(content_image, style_image)[0]

image = tensor_to_image(stylized_image)

cv2.imwrite("image.jpg", image)

if cv2_imshow:
    cv2_imshow(image)
else:
    cv2.imshow(image)